# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else... :/

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
⛰       hills      5
🐊       swamp      7
🌋       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾⛰⛰⛰🌾🌾🌾
🌾🌾⛰⛰🌾🌾🌾
🌾🌾⛰🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌋', '🌋', '🌋', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰'],
['🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾'],
['🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾'],
['🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾'],
['🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾'],
['🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌾', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌲', '🌲', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '⛰', '⛰', '⛰', '⛰', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '🌾', '🐊', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '🌾', '🌾', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌲', '🌲', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['⛰', '🌋', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '⛰', '⛰', '🌋', '🌋', '🌾', '🌋', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊'],
['⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🐊'],
['⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾']
]

In [2]:
for line in full_world:
    print("".join(line))

🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌋🌋🌋🌋🌋🌋🌋🌾🌾
🌾🌾🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰⛰🌋🌋⛰⛰
🌾🌾🌾🌾⛰🌋🌋🌋🌲🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾⛰⛰🌋🌋⛰🌾
🌾🌾🌾⛰⛰🌋🌋🌲🌲🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾⛰🌋🌋🌋⛰🌾
🌾⛰⛰⛰🌋🌋⛰⛰🌾🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾⛰🌋⛰🌾🌾
🌾⛰⛰🌋🌋⛰⛰🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾🌾🌾🌾⛰🌾🌾🌾
🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾⛰⛰⛰🌾🌾
🌾🌾🌾⛰⛰⛰🌾🌾🌾🌾🌾🌾⛰⛰🌋🌋🌾🐊🐊🌾🌾⛰⛰⛰🌾🌾🌾
🌾🌾🌾🐊🐊🐊🌾🌾⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🐊🌾⛰⛰⛰🌾🌾🌾🌾
🌾🌾🐊🐊🐊🐊🐊🌾⛰⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰🌋🌋🌋⛰🌾🌾🌾
🌾🐊🐊🐊🐊🐊🌾🌾⛰🌋🌋⛰🌾🌾🌾🌾🐊🐊🌾🌾⛰🌋🌋⛰🌾🌾🌾
🐊🐊🐊🐊🐊🌾🌾⛰⛰🌋🌋⛰🌾🐊🐊🐊🐊🌾🌾🌾⛰🌋⛰🌾🌾🌾🌾
🌾🐊🐊🐊🐊🌾🌾⛰🌲🌲⛰🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾⛰🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌾🌾🌲🌲🌲🌲⛰⛰⛰⛰🌾🐊🐊🐊🌾🌾⛰🌋⛰🌾🌾🌾
🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰🌾🐊🌾⛰🌋🌋⛰🌾🌾🌾
🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋🌾🌾🌋🌋🌋🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋🌋🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊
🌾🌾⛰⛰⛰⛰🌋🌋🌲🌲🌲🌲🌲🌾🌋🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌲🌲🌋🌋🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🌾⛰⛰🌾🌾🐊🐊🐊🐊🐊🐊
🌾⛰⛰🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋⛰⛰🌾🐊🐊🐊🐊🐊
⛰🌋⛰⛰⛰⛰🌾🌾🌾🌾🌾🌋🌋🌋⛰⛰🌋🌋🌾🌋🌋⛰⛰🐊🐊🐊🐊
⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋🌋🌋⛰⛰⛰⛰🌋🌋🌋🐊🐊🐊🐊
⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾⛰⛰⛰🌾🌾🌾


## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [3]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [4]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [5]:
COSTS = { '🌾': 1, '🌲': 3, '⛰': 5, '🐊': 7}

<div style="background: #4682b4">
    
*JW Note - v2: This COST Dict was revised back to the original state; mountains were removed.*

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
* Do not print raw data structures
* Do not insert unneeded/unrequested spaces
* Use the provided emojis!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [6]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="heuristic"></a>
## heuristic

`heuristic` takes in the current state and goal state, and calculates the euclidean distance between the two points. **Used by**: [get_next_state](#get_next_state)

* **state** Tuple[int,int]: cartesian position of the current state on the map
* **goal**: Tuple[int,int]: cartesian position of the goal state on the map

**returns** euc: Float euclidean distance between state and goal

In [7]:
import numpy as np
def heuristic(state: Tuple[int,int], goal: Tuple[int,int]):
    euc = np.sqrt((state[0] - goal[0])**2 + (state[1] - goal[1])**2)

    return euc

In [8]:
# assertions/unit tests
state = [1,1]
goal = [2,1]
euc = heuristic(state, goal)
assert euc == 1

state = [0,0]
goal = [5,5]
euc = heuristic(state, goal)
assert euc == np.sqrt(50)

state = [5, 6]
goal = [1,2]
euc = heuristic(state, goal)
assert euc == np.sqrt(32)

<a id="Additional Helper Functions"></a>
## Additional Helper Functions

<a id="get_neighbors"></a>
### get_neighbors

`get_neighbors` takes in the world, current state, and possible moves to generate a list of all valid children of the current state. Validity is checked against the bounds of the given world. **Used by**: [a_star_search](#a_star_search)

* **world**: List[List[str]]: Input world map
* **current_state**: Tuple[int,int]: cartesian position of the current state on the map
* **moves**: List[Tuple[int, int]]: list of possible valid moves in cartesian space
* **costs**: Dict[str,int]: dictionary of possible symbols and associated cost to occupy / move

**returns** neighbors: List of valid neighbors (children) of current state in List[Tuple[int,int]] form

In [9]:
def get_neighbors(world: List[List[str]], current_state: Tuple[int, int], moves: List[Tuple[int, int]], 
                  costs: Dict[str, int]): 
    neighbors = []
    for move in moves: 
        x_move, y_move = move[0], move[1]
        test_x, test_y = current_state[0]+x_move, current_state[1]+y_move
        
        if (test_x < 0) or (test_y < 0): #out of bounds - negative
            continue
        if (test_x >= len(world)) or (test_y >= len(world[0])): #out of bounds
            continue
        if world[test_x][test_y] not in costs: #move not allowed
            continue

        neighbor = [test_x, test_y]
        neighbors.append(neighbor)
    
    return neighbors

In [10]:
#assertions / unit tests
world = small_world
moves = MOVES
current_state = [0,0]
neighbors = get_neighbors(world, current_state, moves, COSTS)
assert neighbors == [[1,0], [0,1]]

current_state = [6, 6]
neighbors = get_neighbors(world, current_state, moves, COSTS)
assert neighbors == [[6,5], [5,6]]

current_state = [3,3]
neighbors = get_neighbors(world, current_state, moves, COSTS)
assert neighbors == [[3,2], [4,3], [3,4], [2,3]]

<a id="get_cost"></a>
### get_cost

`get_cost` takes in the world, current state, and cost dictionary to generate the cost of a state in the map. The state (cartesian position) is translated to the world map, and the corresponding symbol is checked in the dictionary. **Used by**: [get_next_state](#get_next_state), [a_star_search](#a_star_search)

* **world**: List[List[str]]: Input world map
* **current_state**: Tuple[int,int]: cartesian position of the current state on the map
* **costs**: Dict[str,int]: dictionary of possible symbols and associated cost to occupy / move

**returns** cost: Integer value of map cost for given location


In [11]:
def get_cost(world: List[List[str]], current_state: Tuple[int, int], costs: Dict[str,int]): 
    cur_x, cur_y = current_state[0], current_state[1]
    symbol = world[current_state[0]][current_state[1]]
    cost = costs[symbol]

    return cost

In [12]:
#assertions / unit tests
world = small_world
costs = COSTS
current_state = [0,0]
cost = get_cost(world, current_state, costs)
assert cost == 1

current_state = [0,1]
cost = get_cost(world, current_state, costs)
assert cost == 3

current_state = [3,0]
cost = get_cost(world, current_state, costs)
assert cost == 1

<a id="on_list"></a>
### on_list

`on_list` takes in a neighbor (state) and a list of positions, and checks if the neighbor is on that list. Used to check states against frontier and explored lists. **Used by**: [a_star_search](#a_star_search)

* **neighbor**: Tuple[int,int]: cartesian position of the neighbor state under consideration
* **list**: List[Tuple[int,int]: list of states to search (such as explored or frontier)

**returns** found: Boolean value for whether the neighbor was found in the given list


In [13]:
def on_list(neighbor: Tuple[int,int], list: List[Tuple[int,int]]): 
    found = False
    for state in list: 
        if state == neighbor: 
            found = True

    return found

In [14]:
#assertions / unit tests
list = [[0,0], [1,2], [3,3], [2, 5]]
neighbor = [0,0]
assert on_list(neighbor, list) == True

neighbor = [3,3]
assert on_list(neighbor, list) == True

neighbor = [2,2]
assert on_list(neighbor, list) == False

<a id="get_next_state"></a>
### get_next_state

`get_next_state` takes in the world, frontier list, costs, goal, and total running cost (g) to evaluate the frontier list for the best next move. This function acts as a priority queue, without reorganizing the frontier list. The output state is the state with the lowest f = g + h value, and can then be removed from the frontier list in the host function. **Used by**: [a_star_search](#a_star_search)

* **world**: List[List[str]]: Input world map
* **frontier**: List[Tuple[int,int]]: List of future states to explore
* **costs**: Dict[str,int]: dictionary of possible symbols and associated cost to occupy / move
* **goal**: Tuple[int,int]: cartesian position of the goal state on the map
* **g**: float: running cost of exploration from start to current point

**returns** best_move: state on the frontier with lowest f value (g+h) 

In [15]:
def get_next_state(world: List[List[str]], frontier: List[Tuple[int,int]], costs: Dict[str, int], 
                   goal: Tuple[int, int], g: float):
    best_move = frontier[0]
    best_f = g + get_cost(world, best_move, costs) + heuristic(best_move, goal) #running cost + next cost + h
    
    for state in frontier: 
        test_f = g + get_cost(world, state, costs) + heuristic(state, goal)
        if test_f < best_f: #new best 
            best_move = state
            best_f = test_f

    return best_move

In [16]:
# assertions / unit tests
world = small_world
costs = COSTS
frontier = [[1,0], [0,1]]
goal = [6,6]
g = 1.0
assert get_next_state(world, frontier, costs, goal, g) == [1,0]

frontier = [[0, 1], [1, 1], [0, 0], [2, 1], [4, 0], [3, 1]]
g = 4.0
assert get_next_state(world, frontier, costs, goal, g) == [3,1]

frontier = [[0, 1], [1, 1], [0, 0], [2, 4], [4, 5], [2, 5], [2, 6], [5, 6]]
g = 10.0
assert get_next_state(world, frontier, costs, goal, g) == [5,6]


<a id="get_direction"></a>
### get_direction

`get_direction` takes in the current state, and next move (state), and outputs a direction in the form of an emoji.  **Used by**: [pretty_print_path](#pretty_print_path)

* **current**: Tuple[int,int]: current state (cartesian)
* **next**: Tuple[int,int]: next state (cartesian)

**returns** emoji: orientation of next move

In [17]:
def get_direction(current: Tuple[int, int], next: Tuple[int, int]): 
    x1, y1 = current[0], current[1]
    x2, y2 = next[0], next[1]
    dy = x2 - x1 #translated for this world
    dx = y2 - y1 #translated for this world
    if dx == 0 and dy > 0: #pos y move
        return '⏬'
    if dx == 0 and dy < 0: #neg y move
        return '⏫'
    if dx > 0 and dy == 0: #pos x move
        return '⏩'
    if dx < 0 and dy == 0: #neg x move
        return '⏪'
    else: 
        return '❌'

In [18]:
# assertions / unit tests
current = [0,0]
next = [1,0]
assert get_direction(current, next) == '⏬'

current = [0,0]
next = [0,1]
assert get_direction(current, next) == '⏩'

current = [0,1]
next = [0,0]
assert get_direction(current, next) == '⏪'

<a id="simple_path"></a>
### simple_path

`simple_path` takes in the path dictionary from `a_star_search` and outputs a simple list of the path. This is the desired path output for the assignment. **Used by**: [a_star_search](#a_star_search)

* **path_dict**: Dict: custom path dictionary generated by `a_star_search`, storing node cost and previous node
* **start**: Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal**: Tuple[int, int]: cartesian position of the goal state on the map


**returns** simple_path: List[Tuple[int, int]] of nodes from start to goal

In [19]:
def simple_path(path_dict: Dict, start: Tuple[int, int], goal: Tuple[int, int]):
    start = [start[0], start[1]] #convert to list
    goal = [goal[0], goal[1]] #convert to list
    # print(path_dict)
    cur_state, simple_path = goal, [goal]  #init
    
    while cur_state != start:
        prev_state = path_dict[str(cur_state)]['prev_pos']
        simple_path.insert(0, prev_state)
        cur_state = prev_state
    
    return simple_path

In [20]:
#Unit Tests / Assertions
dict = {'[1, 0]': {'cost': 1, 'prev_pos': [0, 0]}, '[0, 1]': {'cost': 3, 'prev_pos': [0, 0]}, '[2, 0]': {'cost': 1, 'prev_pos': [1, 0]}, '[1, 1]': {'cost': 3, 'prev_pos': [1, 0]}, '[3, 0]': {'cost': 1, 'prev_pos': [2, 0]}, '[2, 1]': {'cost': 3, 'prev_pos': [2, 0]}, '[4, 0]': {'cost': 3, 'prev_pos': [3, 0]}, '[3, 1]': {'cost': 1, 'prev_pos': [3, 0]}, '[4, 1]': {'cost': 3, 'prev_pos': [3, 1]}, '[3, 2]': {'cost': 1, 'prev_pos': [3, 1]}, '[4, 2]': {'cost': 3, 'prev_pos': [3, 2]}, '[3, 3]': {'cost': 1, 'prev_pos': [3, 2]}, '[2, 2]': {'cost': 3, 'prev_pos': [3, 2]}, '[4, 3]': {'cost': 3, 'prev_pos': [3, 3]}, '[3, 4]': {'cost': 1, 'prev_pos': [3, 3]}, '[2, 3]': {'cost': 3, 'prev_pos': [3, 3]}, '[4, 4]': {'cost': 3, 'prev_pos': [3, 4]}, '[3, 5]': {'cost': 1, 'prev_pos': [3, 4]}, '[2, 4]': {'cost': 3, 'prev_pos': [3, 4]}, '[4, 5]': {'cost': 3, 'prev_pos': [3, 5]}, '[3, 6]': {'cost': 1, 'prev_pos': [3, 5]}, '[2, 5]': {'cost': 3, 'prev_pos': [3, 5]}, '[4, 6]': {'cost': 1, 'prev_pos': [3, 6]}, '[2, 6]': {'cost': 3, 'prev_pos': [3, 6]}, '[5, 6]': {'cost': 1, 'prev_pos': [4, 6]}, '[5, 5]': {'cost': 3, 'prev_pos': [5, 6]}, '[6, 6]': {'cost': 1, 'prev_pos': [5, 6]}}
start = [0,0]
goal = [3,1]
assert(simple_path(dict, start, goal)) == [[0, 0], [1, 0], [2, 0], [3, 0], [3, 1]]

goal = [3, 5]
assert(simple_path(dict, start, goal)) == [[0, 0], [1, 0], [2, 0], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5]]

goal = [6, 6]
assert(simple_path(dict, start, goal)) == [[0, 0], [1, 0], [2, 0], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [4, 6], [5, 6], [6, 6]]

<a id="a_star_search"></a>
## a_star_search

This block uses the A* algorithm to traverse a world map from a start point to a goal. The algorithm initiates a list of frontier and explored positions, evaluates neighbors based on a movement model, chooses a next move (state) based on a cost model (f = g+h). The cost metric f is defined as g (actual running cost from start to current position) + h (euclidean distance from current position to goal). 

Once a next state is chosen from the frontier (in a priority queue fashion, based on f), that state is removed from the frontier. This process continues until a goal is reached, or all states in the frontier have been explored. 

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [21]:
def a_star_search(world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    start, goal = [start[0], start[1]], [goal[0], goal[1]] #convert to list
    frontier, explored, path_dict, g, current_state = [start], [], {}, 0, start #init
    while len(frontier) > 0: 
        prev_state = current_state #store
        current_state = get_next_state(world, frontier, costs, goal, g) #tuple position
        frontier.remove(current_state) #like pop
        cur_cost = get_cost(world, current_state, costs)
        g += cur_cost  #running cost        
        if (current_state[0] == goal[0]) and (current_state[1] == goal[1]): #we're done
            explored.append(current_state)            
            return simple_path(path_dict, start, goal)
        neighbors = get_neighbors(world, current_state, moves, costs)
        for neighbor in neighbors: 
            if (not on_list(neighbor, frontier)) and (not on_list(neighbor, explored)) and (not neighbor == current_state) and (not neighbor == prev_state): #valid move
                frontier.append(neighbor)
                path_dict[str(neighbor)] = {'cost': get_cost(world, neighbor, costs), 'prev_pos': current_state}
        explored.append(current_state)
    return 'NO PATH FOUND' #path not found

In [22]:
# assertions / unit tests
world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]
start = (0, 0)
goal = (len(world[0]) - 1, len(world) - 1)
path = a_star_search(world, start, goal, COSTS, MOVES, heuristic)
# print(path)
assert path == [[0, 0], [1, 0], [2, 0], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [4, 6], [5, 6], [6, 6]]

world = [
    ['🌾', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾']
]
start = [0,0]
goal = [3,3]
path = a_star_search(world, start, goal, COSTS, MOVES, heuristic)
assert path == [[0, 0], [1, 0], [2, 0], [3, 0], [3, 1], [3, 2], [3, 3]]

world = [
    ['🌾', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲'],
    ['⛰', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾']
]
start = [0,0]
goal = [3,3]
path = a_star_search(world, start, goal, COSTS, MOVES, heuristic)
assert path == [[0, 0], [1, 0], [1, 1], [2, 1], [3, 1], [3, 2], [3, 3]]


<a id="pretty_print_path"></a>
### pretty_print_path

`pretty_print_path` takes in the simplified List path output of `simple_path`, determines the proper emoji to indicate direction of motion (along with goal state), and updates the world map with these symbols. This function returns the total path cost. Unit Tests for this code utilizes print statements to visually verify the desired print output.

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

In [23]:
def pretty_print_path(world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], 
                      goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    start = [start[0], start[1]] #convert to list
    goal = [goal[0], goal[1]] #convert to list
    total_cost = 0 #init
    cur_state = path[-1]  #init
    new_world = deepcopy(world) #copy
    for i in range(len(path)-1, 0, -1): 
        cur_state = path[i]
        total_cost += get_cost(world, cur_state, costs)
        if i != 0: 
            prev_state = path[i-1]
            dir_symbol = get_direction(prev_state, cur_state)
            new_world[prev_state[0]][prev_state[1]] = dir_symbol
    new_world[goal[0]][goal[1]] = '🎁' # goal symbol

    for row in new_world: 
        print("".join(row))
    return total_cost # replace with the real value!

In [24]:
# unit tests (visual verification)
world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]
start = (0, 0)
goal = (len(world[0]) - 1, len(world) - 1)
path = a_star_search(world, start, goal, COSTS, MOVES, heuristic)
path_cost = pretty_print_path(world, path, start, goal, COSTS)
print(f"total path cost: {path_cost}")
print('')

world = [
    ['🌾', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾']
]
start = [0,0]
goal = [3,3]
path = a_star_search(world, start, goal, COSTS, MOVES, heuristic)
path_cost = pretty_print_path(world, path, start, goal, COSTS)
print(f"total path cost: {path_cost}")
print('')

world = [
    ['🌾', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲'],
    ['⛰', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾']
]
start = [0,0]
goal = [3,3]
path = a_star_search(world, start, goal, COSTS, MOVES, heuristic)
path_cost = pretty_print_path(world, path, start, goal, COSTS)
print(f"total path cost: {path_cost}")

⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
total path cost: 12

⏬🌲🌲🌲
⏬🌲🌲🌲
⏬🌲🌲🌲
⏩⏩⏩🎁
total path cost: 6

⏬🌲🌲🌲
⏩⏬🌲🌲
⛰⏬🌲🌲
🌾⏩⏩🎁
total path cost: 10


## Problems

## Problem 1 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [25]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
total path cost: 12
[[0, 0], [1, 0], [2, 0], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [4, 6], [5, 6], [6, 6]]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [26]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
⏩⏬🌾⏩⏩⏩⏬🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌋🌋🌋🌋🌋🌋🌋🌾🌾
🌾⏩⏩⏫🌋🌋⏩⏩⏬🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰⛰🌋🌋⛰⛰
🌾🌾🌾🌾⛰🌋🌋🌋⏬🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾⛰⛰🌋🌋⛰🌾
🌾🌾🌾⛰⛰🌋🌋🌲⏬🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾⛰🌋🌋🌋⛰🌾
🌾⛰⛰⛰🌋🌋⛰⛰⏬🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾⛰🌋⛰🌾🌾
🌾⛰⛰🌋🌋⛰⛰🌾⏬🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾🌾🌾🌾⛰🌾🌾🌾
🌾🌾⛰⛰⛰⛰⛰🌾⏬🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾⛰⛰⛰🌾🌾
🌾🌾🌾⛰⛰⛰🌾⏬⏪🌾🌾🌾⛰⛰🌋🌋🌾🐊🐊🌾🌾⛰⛰⛰🌾🌾🌾
🌾🌾🌾🐊🐊🐊🌾⏬⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🐊🌾⛰⛰⛰🌾🌾🌾🌾
🌾🌾🐊🐊🐊🐊🐊⏬⛰⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰🌋🌋🌋⛰🌾🌾🌾
🌾🐊🐊🐊🐊🐊⏬⏪⛰🌋🌋⛰🌾🌾🌾🌾🐊🐊🌾🌾⛰🌋🌋⛰🌾🌾🌾
🐊🐊🐊🐊🐊🌾⏬⛰⛰🌋🌋⛰🌾🐊🐊🐊🐊🌾🌾🌾⛰🌋⛰🌾🌾🌾🌾
🌾🐊🐊🐊🐊🌾⏬⛰🌲🌲⛰🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾⛰🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌾⏩⏩⏩⏩⏬⛰⛰⛰⛰🌾🐊🐊🐊🌾🌾⛰🌋⛰🌾🌾🌾
🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲⏩⏬🌋🌋🌋⛰⛰🌾🐊🌾⛰🌋🌋⛰🌾🌾🌾
🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲⏩⏩⏬🌋🌋🌋🌾🌾🌋🌋🌋🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲⏩⏬🌲🌋🌋🌋🌋🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲⏩⏩⏬⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌾🌾🌾⏩⏩⏩⏬🌾🌾🌾🐊🐊🐊🐊
🌾🌾⛰⛰⛰⛰🌋🌋🌲🌲🌲🌲🌲🌾🌋🌾🌾🌾🌾⏬🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌲🌲🌋🌋🌾🌾🌾🌾🌾⏬🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🌾⛰⛰⏩⏬🐊🐊🐊🐊🐊🐊
🌾⛰⛰🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋⛰⏩⏬🐊🐊🐊🐊🐊
⛰🌋⛰⛰⛰⛰🌾🌾🌾🌾🌾🌋🌋🌋⛰⛰🌋🌋🌾🌋🌋⏩⏩⏩⏬🐊🐊
⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋🌋🌋⛰⛰⛰⛰🌋🌋🌋🐊⏬🐊🐊
⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾⛰⛰⛰⏩⏩🎁
total path cost: 120
[[0, 0], [1, 0], [1, 1], [2, 1], [2, 2], [2, 3], [1, 3], [1, 4], [1, 5], [1, 6], [2, 6], [2, 7], [2, 8], [3, 8], [4, 8], [5, 8], [6, 8], [7, 8], [8, 8], [8, 7], [9, 7], [10, 7], [11, 7], [11, 6], [12, 6], [13, 6], [14, 6], 

<a id="Comments"></a>
## Comments from Original Submittal

(This is the place to leave me comments)
- COSTS was updated to include the mountain; value set to 10000 as a high marker rather than np.inf
- I think my approach of outputting the explored list is incorrect - it works on the small world, but not on the full map. The path shows all explored nodes, including those that resulted in a deadend. The red X markers show where my direction function threw an error, usually because states were not progressing cleanly.
- I believe I need to track parents / last position as I progress through the tree...but I wasn't sure how to do this without using what is essentially object oriented data structures. In other courses I would have used a custom "node" class with a "node.parent" parameter, essentially building out a tree as I go. In this case, I tried a dictionary with individual keys for each node, but was unsuccessful in my implementation. I'm interested in hearing your thoughts / solution on this! 

<a id="Notes for Resubmit"></a>
## Comments for Resubmit

**Grading Feedback**
- *Setting mountains to a really high cost isn't always going to work. - For example, if you had mountains that were diagonal across a 10000 x 1000 map except for the the left most bottom, a_star would have you navigate over the mountains when that's not allowed. Better to just add logic in the generate succesors method to just not allow the move*
- *a_star_search over 20 lines*
- *As you said in your comments, your implementation is slightly off. The optimal cost is just under 100 for the hard problem*
- *To answer your comment: you can use a dict as a place holder for a class. I know this doesn't feel much different than just making a node class, but for the class, its good enough. - So, in your dict, you'd have something like - dict( (x,y) = dict( cost=value_1, prev_node=(a,b), ... ) (x+1,y) = dict( cost=value_2, prev_node=(c,b), ... ) ... )*

**Changes Made in this Resubmit**
* Cost Dict was revised back to original - mountains were removed
* `get_neighbors` function updated to intake costs, check possible neighbors against cost dictionary; if not in Dict, move is not allowed, and location is not included in neighbors. All necessary function calls were updated to pass the costs dict.
* `path_cost` helper function no longer needed - eliminated
* `a_star_search` updated logic to build a path dictionary, storing each node's cost and previous connecting node; this dictionary was passed to a new function `simple_path` to output the path in the desired format. Free lines reduced to get total count < 20 (this makes the function less readable, but meets requirements).
* `simple_path` function added to translate path dictionary output from `a_star_search` into the assignment's desired list format
* `pretty_print_path` updated to print from output of `simple_path`

**Additional Notes** 
* A few questions were sent to Dr. Strasser ~48 hours prior to the resubmit deadline regarding the output format, and total path cost. No answers were received prior to this resubmittal, so assumptions were made around the creation of `simple_path` and the total path cost shown above.
* In office hours, it was stated the optimal path should track the left boundary down, then snake across to the goal on the right. My emailed questions were targeting this discrepancy... I'm not sure what is causing my math to drive me in a diagonal fashion towards the goal. I'm interested in your thoughts here. 

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.